In [ ]:
from langchain_aws import BedrockLLM
from langchain_aws.agents.base import BedrockAgentsRunnable
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompt_values import PromptValues
import boto3
import uuid
import config

In [76]:
session = boto3.Session(
    profile_name=config.aws_profile,
    region_name=config.aws_region)

boto3_client = session.client('bedrock-agent-runtime', region_name=config.aws_region)
boto3_runtime = session.client('bedrock-runtime', region_name=config.aws_region)

agent = BedrockAgentsRunnable(
    agent_alias_id=config.agent_alias_id,
    agent_id=config.agent_id,
    region_name=config.aws_region,
    client=boto3_client
)

In [69]:
response = agent.invoke(
    {
        "input": "What do you know about hazard and safety? Give reference of the material that you get from knowledge base",
        "session_id": str(uuid.uuid4())
    }
)

In [77]:
response_schemas = [
    ResponseSchema(name="Quote", description="Phrase from the knowledge base sources"),
    ResponseSchema(name="Source", description="Source of the quote"),
]

In [78]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [79]:
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

In [87]:
llm = BedrockLLM(
    credentials_profile_name=config.aws_profile,
    model=config.model_id
)

In [88]:
response = llm.invoke(input="What's up")
print(response)

ERROR:root:Error raised by bedrock service: An error occurred (ValidationException) when calling the InvokeModel operation: Malformed input request: #: required key [prompt] not found#: required key [max_tokens_to_sample] not found#: extraneous key [inputText] is not permitted, please reformat your input and try again.


ValidationException: An error occurred (ValidationException) when calling the InvokeModel operation: Malformed input request: #: required key [prompt] not found#: required key [max_tokens_to_sample] not found#: extraneous key [inputText] is not permitted, please reformat your input and try again.